Index: Speed up a query!

Allows me to retrieve data more efficiently (if used correctly) becouse the queries only have to look at a subset of all documents

* It's a list of values specified on the 1st parameter < {} > (Works like find / update search paramenter ) and pointers to the original document
* Could be text index or number index
* Cons: They will slow down my writes

In [ ]:
#       < 1 > Creates an index with ascending sorting. < -1 > descending sorting
> db.collectionName.createIndex({value: -1}}) 
#i.e. 
> db.contacts.createIndex({"dob.age": 1}}) 

#       See all indexes that exist for a collection (always gonna be a default _id: index)
> db.collectionName.getIndexes()

#       Drop index: Should specify the exact same document used for creating the Index
# get Indexes show me the name of the index created. I can use that inf to drop a specific index
> db.collectionName.dropIndex({value: 1}) 
#i.e. 
> db.contacts.dropIndex({"dob.age": 1}) 

#       Compound indexes use more than one doc field to create the list. Go from left to right. 
#Takes first all docs that match firstKey and from those, takes the ones that matchs secondKey
> db.collectionName.createIndex({fisrtKey: 1, secondKey: 1}) 
#i.e. Case of SUCCESFUL MULTI-KEY index: puedo indexar un campo que sea un array (JUST ONE)
# en este caso tomo todos los nombres y de esos nombres organizo el array de hobbies
> db.collectionName.createIndex({name: 1, hobbies: 1})
#i.e. ERROR MULTI-KEY index: intento indexar dos campos que son arrays o colección de documentos
# cannot index parallel arrays! Se repetiría el almacenamiento de hobbies en tantos elementos tenga favCars
> db.collectionName.createIndex({favCars: 1, hobbies: 1})



#       Configure index (1st parameter) with a second parameter.
> db.collectionName.createIndex({value: 1} , {configure index})
#i.e. Index of emails in ascending order and should be a unique value. So cannot exist repeated emails or will show an error
> db.contacts.createIndex({"email": 1}, {unique: true}) 


#       Partial filters works kinda like configure index. 
#Just gonna create an index of the specified values in 1st parameter if fits with conditions from second paramenter.
> db.collectionName.createIndex({saveValue: 1}, {partialFilterExpression: {field: "value"}}) 
#i.e. Create an index of dob.age only witch the docs where gender is male. 
> db.contacts.createIndex({"dob.age": 1}, {partialFilterExpression: {gender: "male"}}) 
# if i wanna access to this index i must set the partial filter expression. 
# it doesn't matter the order, just matters put in <find> all the fields used in the index
#i.e. from the index list i wanna grab all the male with age greater than 30
> db.contacts.findMany({"dob.age": {$gte: 30}, gender: "male"})


> db.collectionName.insertOne({name: "ThisName", createdAt: new Date()}) #current timestamp of current date
#       Time to live (TTL) index. Helpful when i have self-destroying data. Como iniciar sesión y después de un rato expira
# This index only works with DATE INDEXES or DATE FIELDS
# Los documentos insertados después de que se creo el index van a eliminarse después de 10 segs
# Maintain a collection of documents which destroy themselves after a certain time span
> db.collectionName.createIndex({createdAt: 1}, {expireAfterSeconds: 10})






In [ ]:
#       Text indexes: Se hace como con palabras clave
#Definir el campo con strings al que le quiero crear el index. 
# En vez de decir que quiero organizarlo ascendente o descendente. Digo que es de tipo <"text">.
> db.collectionName.createIndex({stringField: "text"})
> db.collectionName.createIndex({stringField: "text", secondField: "text"}) #conbined text indexes
# This allows me to use the special <$text> key to search inside the string of the field declared 
> db.collectionName.find({$text: {$search: "\"this phrase\""}}) #this sintax allows to match a phrase. Not individual terms
> db.collectionName.find({$text: {$search: "singleWord"}})
> db.collectionName.find({$text: {$search: "word1 word2"}}) #gonna match like individual terms
> db.collectionName.find({$text: {$search: "word1 -word2"}}) #the minus < - > excluded the second word in the query.


#       Setting default language (theres only like 15)
#       Weights: more relevant find X word in field1 than field2. In this case secondField worth 12 times more than stringField
> db.collectionName.createIndex({stringField: "text", secondField: "text"}, {default_language: "spanish", weights:{stringField: 1, secondField: 12}})




